# 第8章: ニューラルネット
第6章で取り組んだニュース記事のカテゴリ分類を題材として，ニューラルネットワークでカテゴリ分類モデルを実装する．なお，この章ではPyTorch, TensorFlow, Chainerなどの機械学習プラットフォームを活用せよ．

## 70. 単語ベクトルの和による特徴量
問題50で構築した学習データ，検証データ，評価データを行列・ベクトルに変換したい．例えば，学習データについて，すべての事例$x_i$の特徴ベクトル$x_i$を並べた行列$X$と，正解ラベルを並べた行列（ベクトル）$Y$を作成したい．
$$$
X = \begin{pmatrix} 
  \boldsymbol{x}_1 \\ 
  \boldsymbol{x}_2 \\ 
  \dots \\ 
  \boldsymbol{x}_n \\ 
\end{pmatrix} \in \mathbb{R}^{n \times d},
Y = \begin{pmatrix} 
  y_1 \\ 
  y_2 \\ 
  \dots \\ 
  y_n \\ 
\end{pmatrix} \in \mathbb{N}^{n}
$$$
ここで，$n$は学習データの事例数であり，$x_i \in R^d$とyi∈ℕはそれぞれ，i∈{1,…,n}番目の事例の特徴量ベクトルと正解ラベルを表す． なお，今回は「ビジネス」「科学技術」「エンターテイメント」「健康」の4カテゴリ分類である．ℕ<4で4未満の自然数（0を含む）を表すことにすれば，任意の事例の正解ラベルyiはyi∈ℕ<4で表現できる． 以降では，ラベルの種類数をLで表す（今回の分類タスクではL=4である）．

i番目の事例の特徴ベクトルxiは，次式で求める．

$$
\boldsymbol{x}_i = \frac{1}{T_i} \sum_{t=1}^{T_i} \mathrm{emb}(w_{i,t})
$$
ここで，i番目の事例はTi個の（記事見出しの）単語列(wi,1,wi,2,…,wi,Ti)から構成され，emb(w)∈ℝdは単語wに対応する単語ベクトル（次元数はd）である．すなわち，i番目の事例の記事見出しを，その見出しに含まれる単語のベクトルの平均で表現したものがxiである．今回は単語ベクトルとして，問題60でダウンロードしたものを用いればよい．300次元の単語ベクトルを用いたので，d=300である．

i番目の事例のラベルyiは，次のように定義する．
$$
% <![CDATA[
y_i = \begin{cases}
0 & (\mbox{記事}x_i\mbox{が「ビジネス」カテゴリの場合}) \\
1 & (\mbox{記事}x_i\mbox{が「科学技術」カテゴリの場合}) \\
2 & (\mbox{記事}x_i\mbox{が「エンターテイメント」カテゴリの場合}) \\
3 & (\mbox{記事}x_i\mbox{が「健康」カテゴリの場合}) \\
\end{cases} %]]>
$$

なお，カテゴリ名とラベルの番号が一対一で対応付いていれば，上式の通りの対応付けでなくてもよい．

以上の仕様に基づき，以下の行列・ベクトルを作成し，ファイルに保存せよ．

- 学習データの特徴量行列: Xtrain∈ℝNt×d
- 学習データのラベルベクトル: Ytrain∈ℕNt
- 検証データの特徴量行列: Xvalid∈ℝNv×d
- 検証データのラベルベクトル: Yvalid∈ℕNv
- 評価データの特徴量行列: Xtest∈ℝNe×d
- 評価データのラベルベクトル: Ytest∈ℕNe

なお，Nt,Nv,Neはそれぞれ，学習データの事例数，検証データの事例数，評価データの事例数である．

In [35]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import numpy as np

# word2becモデル読み込み
model = KeyedVectors.load_word2vec_format('files/GoogleNews-vectors-negative300.bin', binary=True)


from functools import lru_cache

@lru_cache(maxsize=None)
def get_vector(word):
    return model.get_vector(word)

def sentence2vec(text):
    """
    文字列を単語分割→ word2vecでベクトル化した平均を取る
    英単語想定
    """
    # 前処理
    preprocessed_text = text.replace('"','').replace('-','').replace(';','').replace(',','').replace('.','')
    words = preprocessed_text.split(' ')
    result_vec = np.zeros((1,300))
    
    n_error = 0
    for word in words:
        try:
#             result_vec += model.get_vector(word)
            result_vec += get_vector(word)
        except:
            n_error += 1
            
    result_vec = result_vec/n_valid_words if (n_valid_words :=(len(words) - n_error)) != 0 else result_vec
    return result_vec

In [36]:
%%time
import csv



train_path = 'files/train.txt'
train_y = []
train_X = []

with open(train_path) as f:
    reader = csv.DictReader(f,fieldnames=['label','sentense'],delimiter='\t')
    for row in reader:
        train_y.append(row['label'])
        train_X.append(sentence2vec(row['sentense']))

train_y = np.array(train_y)
train_X = np.vstack(train_X)
print(f'train_y:{train_y.shape}\ntrain_X:{train_X.shape}')
    



train_y:(10684,)
train_X:(10684, 300)
CPU times: user 1.3 s, sys: 1.13 s, total: 2.43 s
Wall time: 4.26 s


In [37]:
%%time 

valid_path = 'files/valid.txt'
valid_y = []
valid_X = []

with open(valid_path) as f:
    reader = csv.DictReader(f,fieldnames=['label','sentense'],delimiter='\t')
    for row in reader:
        valid_y.append(row['label'])
        valid_X.append(sentence2vec(row['sentense']))

valid_y = np.array(valid_y)
valid_X = np.vstack(valid_X)
print(f'valid_y:{valid_y.shape}\nvalid_X:{valid_X.shape}')

valid_y:(1336,)
valid_X:(1336, 300)
CPU times: user 132 ms, sys: 46.1 ms, total: 178 ms
Wall time: 527 ms


In [38]:
%%time
import csv



test_path = 'files/test.txt'
test_y = []
test_X = []

with open(test_path) as f:
    reader = csv.DictReader(f,fieldnames=['label','sentense'],delimiter='\t')
    for row in reader:
        test_y.append(row['label'])
        test_X.append(sentence2vec(row['sentense']))

test_y = np.array(test_y)
test_X = np.vstack(test_X)
print(f'test_y:{test_y.shape}\ntest_X:{test_X.shape}')



test_y:(1336,)
test_X:(1336, 300)
CPU times: user 116 ms, sys: 46.5 ms, total: 163 ms
Wall time: 285 ms


## 71. 単層ニューラルネットワークによる予測
問題70で保存した行列を読み込み，学習データについて以下の計算を実行せよ．

$$ \hat{\boldsymbol{y}}_1 = {\rm softmax}(\boldsymbol{x}_1 W),$$
$$ \hat{Y} = {\rm softmax}(X_{[1:4]} W)$$ 

ただし，softmax
はソフトマックス関数，$X_{[1:4]} \in \mathbb{R}^{4 \times d}$
は特徴ベクトル$x_1,x_2,x_3,x_4$
を縦に並べた行列である．

行列$W \in \mathbb{R}^{d \times L}$
は単層ニューラルネットワークの重み行列で，ここではランダムな値で初期化すればよい（問題73以降で学習して求める）．なお，$\hat{\boldsymbol{y}}_1 \in \mathbb{R}^L$
は未学習の行列$W$
で事例$x_1$
を分類したときに，各カテゴリに属する確率を表すベクトルである． 同様に，$ \hat{Y} \in \mathbb{R}^{n \times L}$
は，学習データの事例$x_1,x_2,x_3,x_4$
について，各カテゴリに属する確率を行列として表現している．



## 72. 損失と勾配の計算
学習データの事例$x_1$
と事例集合$x_1,x_2,x_3,x_4$
に対して，クロスエントロピー損失と，行列$W$
に対する勾配を計算せよ．なお，ある事例$x_i$
に対して損失は次式で計算される．

$$ l_i = - \log [\mbox{事例}x_i\mbox{が}y_i\mbox{に分類される確率}]$$

ただし，事例集合に対するクロスエントロピー損失は，その集合に含まれる各事例の損失の平均とする．

## 73. 確率的勾配降下法による学習
確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，行列$W$
を学習せよ．なお，学習は適当な基準で終了させればよい（例えば「100エポックで終了」など）．

## 74. 正解率の計測
問題73で求めた行列を用いて学習データおよび評価データの事例を分類したとき，その正解率をそれぞれ求めよ．

## 75. 損失と正解率のプロット
問題73のコードを改変し，各エポックのパラメータ更新が完了するたびに，訓練データでの損失，正解率，検証データでの損失，正解率をグラフにプロットし，学習の進捗状況を確認できるようにせよ．

## 76. チェックポイント
問題75のコードを改変し，各エポックのパラメータ更新が完了するたびに，チェックポイント（学習途中のパラメータ（重み行列など）の値や最適化アルゴリズムの内部状態）をファイルに書き出せ．

## 77. ミニバッチ化
問題76のコードを改変し，B
事例ごとに損失・勾配を計算し，行列$W$
の値を更新せよ（ミニバッチ化）．B
の値を1,2,4,8,…
と変化させながら，1エポックの学習に要する時間を比較せよ．

## 78. GPU上での学習
問題77のコードを改変し，GPU上で学習を実行せよ．

## 79. 多層ニューラルネットワーク
問題78のコードを改変し，バイアス項の導入や多層化など，ニューラルネットワークの形状を変更しながら，高性能なカテゴリ分類器を構築せよ．